In [ ]:
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset


In [0]:
# 读取时间序列数据
data = pd.read_csv('time.csv')

# 假设你的数据有一个特征列，列名为 'value'，表示时间序列的值
time_series = data['value'].values

# 将时间序列数据转换为PyTorch张量
time_series_tensor = torch.tensor(time_series, dtype=torch.float32)

# 定义窗口大小（即每个输入序列的长度）
window_size = 10

# 构建输入和目标序列
inputs = []
targets = []

for i in range(len(time_series) - window_size):
    inputs.append(time_series_tensor[i:i+window_size])
    targets.append(time_series_tensor[i+window_size])

# 将输入和目标序列转换为PyTorch张量
inputs = torch.stack(inputs)
targets = torch.stack(targets)

# 创建数据集和数据加载器
dataset = TensorDataset(inputs, targets)
batch_size = 64
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


In [ ]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        
        out, _ = self.lstm(x, (h0, c0))
        
        # 取最后一个时间步的输出作为模型输出
        out = self.fc(out[:, -1, :])
        return out

# 假设输入时间序列的维度为1，输出维度也为1
input_size = 1
output_size = 1
hidden_size = 32
num_layers = 2

# 创建LSTM模型
model = LSTMModel(input_size, hidden_size, num_layers, output_size)

# 定义损失函数和优化器
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [ ]:
# 初始化模型
input_size = 1
output_size = 1
hidden_size = 32
num_layers = 2

model = LSTMModel(input_size, hidden_size, num_layers, output_size)

# 定义损失函数和优化器
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# 准备数据加载器
dataset = TensorDataset(inputs, targets)
batch_size = 64
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# 训练模型
num_epochs = 100
for epoch in range(num_epochs):
    for batch_inputs, batch_targets in dataloader:
        # 前向传播
        outputs = model(batch_inputs)
        loss = criterion(outputs, batch_targets)
        
        # 反向传播和优化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    # 每个epoch结束后打印损失
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# 预测未来的值
future = 30
preds = []
model.eval()
with torch.no_grad():
    last_seq = inputs[-1].unsqueeze(0)  # 取最后一个输入序列
    for _ in range(future):
        pred = model(last_seq)
        preds.append(pred.item())
        last_seq = torch.cat([last_seq[:, 1:], pred.unsqueeze(1)], axis=1)

# 打印预测结果
print("Predictions for the next", future, "time steps:")
print(preds)